In [1]:
import pandas as pd
import numpy as np
from random import gauss
from math import exp, sqrt
import math
import datetime
from numpy import *
import matplotlib.pyplot as plt
import collections
from WindPy import w
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [2]:
# class TreeNode:
#     def __init__(self, stockPrice = 0, left = None, right = None,leftParent = None, rightParent = None):
#         self.stockPrice = stockPrice
#         self.left = left
#         self.right = right
#         self.leftParent = leftParent
#         self.rightParent = rightParent

In [3]:
# node1 = TreeNode(1)
# node2 = TreeNode(2)
# node4 = TreeNode(5)
# node5 = TreeNode(6)
# node6 = TreeNode(right = node1)
# node6.right.val

In [4]:
def getFactors(riskFreeRate, dt, volatility):
    """
    计算标的价格向上，向下走的的两个价格因子
    返回list
    """
    return [math.exp(riskFreeRate * dt + volatility * sqrt(dt)), math.exp(riskFreeRate * dt - volatility * sqrt(dt))]

In [5]:
def getProbabilities(riskFreeRate, dt, factors):
    """
    计算标的价格向上，向下走的概率
    返回list
    """
    goUp = (math.exp(riskFreeRate * dt) - factors[1]) / (factors[0] - factors[1])
    goDown = 1 - goUp
    return [goUp, goDown]

In [6]:
def getStockPriceAtEachNode(initialStockPrice, tradingDays):
    """
    计算每一个节点上的标的价格（根据价格因子计算）
    例，初始价1，向上向下因子为1.2和0.8，则下一阶段的两个方向走势价格为1.2*1和0.8*1
    """
    allResults = []
    rootLevel = [{"标的价格":initialStockPrice}]
    allResults.append(rootLevel)
    previousLevel = rootLevel
    for level in range(1, tradingDays):
        currentLevelResult = generateOneLevelResult(previousLevel, goUpFactor, goDownFactor)
        allResults.append(currentLevelResult)
        previousLevel = currentLevelResult
    return allResults

In [7]:
def generateOneLevelResult(previousLevel, upFactor, downFactor):
    """
    计算每一个节点上的标的价格（根据价格因子计算），根据前一层价格计算下一层价格
    例，初始价1，向上向下因子为1.2和0.8，则下一阶段的两个方向走势价格为1.2*1和0.8*1
    """
    result = []
    result.append({"标的价格":previousLevel[0]["标的价格"] * downFactor})
    for i in range(0, len(previousLevel)):
        childValue = previousLevel[i]["标的价格"] * upFactor
        result.append({"标的价格":childValue})
    return result

In [8]:
def calculatePayOffAtEachNode(strikingPrice, optionType, tree):
    """
    计算每一个node上的payoff
    payoff计算公式：payoff（put）= max(strike - price, 0)
                  payoff(call) = max(price - strike, 0)
    """
    
    for i in range(0, len(tree)):
        for node in tree[i]:
            if(optionType == "CALL"):
                payOff = max(node["标的价格"] - strikingPrice, 0)
            else:
                payOff = max(strikingPrice - node["标的价格"], 0)
            node["payoff"] = payOff

In [9]:
def calculateMaxPayoffAtEachNode(riskFreeRate, dt, goUpProb, goDownProb, tree):
    """
    计算每一个节点上的最大payoff
    计算逻辑：比较当前节点payoff，与下一个时间节点上产生的向上与向下价格走势节点上的payoff按向上向下概率的权重计算加总，再折线到当前时间节点，的较大者
            即当前节点行权的收益，与当前不行权，选择到下一个节点再行权的收益折线，两者较大者，为当前节点的最大收益。
    计算公式：MAX(当前节点payoff,(下一个时间点价格向上走的payoff*价格增长的概率+下一个时间点价格向下走的payoff*标的价格下降的概率)*EXP(-无风险利率*1/tradingdays-1))
    """
    numberOfLevel = len(tree)
    for node in tree[len(tree) - 1]:
        node["最大回报"] = node["payoff"]
    
    for level in range(numberOfLevel-1, -1, -1):
        currentLevel = tree[level]
        for i in range(0, len(currentLevel)):
            if(level == numberOfLevel - 1):
                maxPayoff = currentLevel[i]["payoff"]
            else:
                oneLevelDownList = tree[level + 1]
                maxPayoff = max(currentLevel[i]["payoff"], 
                                (oneLevelDownList[i]["最大回报"] * goDownProb + oneLevelDownList[i + 1]["最大回报"] * goUpProb) * exp(-riskFreeRate * dt))
            currentLevel[i]["最大回报"] = maxPayoff

In [10]:
def printAll(result):
    for i in range(0, len(result)):
        print("第" + str(i) + "行")
        for j in result[i]:
            print(j)

In [11]:
"""参数设置"""
INITIAL_STOCK_PRICE = 1
VOLATILITY = 0.3
RISK_FREE_RATE = 0.05
STRIKING_PRICE = 1.05
TRADING_DAYS = 5
NUMBER_OF_PERIOD = TRADING_DAYS - 1
dt = 1/NUMBER_OF_PERIOD
OPTION_TYPE = "CALL"

factors = getFactors(RISK_FREE_RATE, dt, VOLATILITY)
goUpFactor = factors[0]
goDownFactor = factors[1]

probabilities = getProbabilities(RISK_FREE_RATE, dt, factors)
goUpProb = probabilities[0]
goDownProb = probabilities[1]

"""建立二叉树，计算每个node上的标的价格"""
tree = getStockPriceAtEachNode(INITIAL_STOCK_PRICE, TRADING_DAYS)

"""计算每个node上的payoff"""
calculatePayOffAtEachNode(STRIKING_PRICE, OPTION_TYPE, tree)

"""计算每个node上的最大payoff"""
calculateMaxPayoffAtEachNode(RISK_FREE_RATE, dt, goUpProb, goDownProb, tree)
# printAll(tree)
print("二叉树方法美式期权定价结果：" + str(tree[0][0]["最大回报"]))

二叉树方法美式期权定价结果：0.11284074749471591


In [16]:
# root = TreeNode(INITIAL_STOCK_PRICE)
# root.left = TreeNode(root.stockPrice * goDownFactor, rightParent = root)
# root.right = TreeNode(root.stockPrice * goUpFactor, leftParent = root)

# #初始化一个queue，把二叉树第二层两个数放进去
# queue = collections.deque()
# queue.append(root.left)
# queue.append(root.right)

# numberOfParent = len(queue)

# #level：parent所在的层数， 第n层有n个parent node
# # for level in range(2, TRADING_DAYS):
# firstNodeInLevel = queue.popleft()

# #生成第一个子节点
# childNode1 = TreeNode(firstNodeInLevel.stockPrice * goDownFactor)
# queue.appendleft(firstNodeInLevel)

# #生成其他的子节点, 当前层为level，即有level个parent，要生成一共（level+1）个子节点，已生成2个,还需要生成（level-1）个
# previousNode = childNode1

# numberOfChildGenerated = 1
# while(len(queue) < TRADING_DAYS):
#     print(len(queue))
    
#     currentParentNode = queue.popleft()
#     newChildNode = TreeNode(currentParentNode.stockPrice * goUpFactor, leftParent = currentParentNode)
#     numberOfChildGenerated += 1
    
#     currentParentNode.left = previousNode
#     currentParentNode.right = newChildNode
#     previousNode.rightParent = currentParentNode
    
#     queue.append(previousNode)
#     previousNode = newChildNode
        

In [15]:

import matplotlib.pyplot as plt
import statsmodels.api as sm 

X = [1.08, 1.26, 1.07, 1.97, 1.56, 0.77, 0.84, 1.22]
y = [0, 0, 0.0659232,0.1695168, 0, 0.188352, 0.0847584, 0]
model = sm.OLS(y,X)
results = model.fit()
results.summary()

C:\Users\wangsu\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:1394: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.391
Model:                            OLS   Adj. R-squared:                  0.304
Method:                 Least Squares   F-statistic:                     4.500
Date:                Sun, 07 Feb 2021   Prob (F-statistic):             0.0716
Time:                        17:14:21   Log-Likelihood:                 9.2735
No. Observations:                   8   AIC:                            -16.55
Df Residuals:                       7   BIC:                            -16.47
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0477      0.023      2.121      0.072      -0.005       0.101
==============================================================================
Omnibus:                        1.250   Durbin-Watson:                   2.277
Prob(Omnibus):                  0.535   Jarque-Bera (JB):                0.822
Skew:                           0.666   Prob(JB):                        0.663
Kurtosis:                       2.168   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""